# Final Project
# Rohith Perumandla
# ID: 2174629

# Installation and Importing Libraries

In [1]:
# Installing the required packages
sc.install_pypi_package("numpy")
sc.install_pypi_package("matplotlib")
sc.install_pypi_package("seaborn")
sc.install_pypi_package("sklearn")
sc.install_pypi_package("IPython")
sc.install_pypi_package("boto3")
sc.install_pypi_package("python-dateutil==2.8.2")


VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1742356023609_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…



  Attempting uninstall: python-dateutil
    Found existing installation: python-dateutil 2.8.1
    Not uninstalling python-dateutil at /usr/lib/python3.9/site-packages, outside environment /mnt1/yarn/usercache/livy/appcache/application_1742356023609_0001/container_1742356023609_0001_01_000001/tmp/spark-8afa3b55-4db8-42e3-8bd0-7b82b49560e2
    Can't uninstall 'python-dateutil'. No files were found to uninstall.

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'
  Preparing metadata (setup.py): started
  Preparing metadata (setup

# Accessing the Books data from S3 Bucket

In [ ]:
from pyspark.sql import SparkSession

# Initializing Spark session with S3 configurations
spark = SparkSession.builder \
    .appName("BookRecommendation") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.fs.s3a.access.key", "your key") \
    .config("spark.hadoop.fs.s3a.secret.key", "Your key") \
    .config("spark.hadoop.fs.s3a.endpoint", "s3.amazonaws.com") \
    .getOrCreate()

# Load data from S3
df_ratings = spark.read.json("s3://book-recommendation-project/ratings.json")
df_books = spark.read.json("s3://book-recommendation-project/metadata.json")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Data Cleaning and Transformation

In [4]:
df_ratings.show(5)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+------+-------+
| item_id|rating|user_id|
+--------+------+-------+
|41335427|     5|      0|
|41335427|     3|      1|
|41335427|     5|      2|
|41335427|     5|      3|
|41335427|     5|      4|
+--------+------+-------+
only showing top 5 rows

In [6]:
df_books.show(5)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+--------------------+--------------------+--------+----+--------------------+--------------------+----+
|        authors|         description|                 img| item_id|lang|               title|                 url|year|
+---------------+--------------------+--------------------+--------+----+--------------------+--------------------+----+
|     John Green|There is an alter...|https://images.gr...|16827462| eng|The Fault in Our ...|https://www.goodr...|2012|
|Suzanne Collins|Winning will make...|https://images.gr...| 2792775| eng|The Hunger Games ...|https://www.goodr...|2008|
|Suzanne Collins|My name is Katnis...|https://images.gr...| 8812783| eng|Mockingjay (The H...|https://www.goodr...|2010|
|  Paula Hawkins|Every day the sam...|https://images.gr...|41107568| eng|The Girl on the T...|https://www.goodr...|2015|
|Suzanne Collins|Sparks are igniti...|https://images.gr...| 6171458| eng|Catching Fire (Th...|https://www.goodr...|2009|
+---------------+---------------

In [7]:
print("Total Number of Rows in the Books dataset:",df_books.count())
print("Total Number of Rows in the Ratings dataset:",df_ratings.count())


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Total Number of Rows in the Books dataset: 9374
Total Number of Rows in the Ratings dataset: 5152656

In [8]:
# Example transformation (filtering valid ratings)
df_ratings_clean = df_ratings.filter(df_ratings['rating'].isNotNull())
print("Total Number of Rows in the Ratings in the dataset after cleaning:",df_ratings.count())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Total Number of Rows in the Ratings in the dataset after cleaning: 5152656

In [10]:
from pyspark.sql.functions import col

# Remove duplicate entries
df_ratings_clean = df_ratings.dropDuplicates()

# Filter invalid ratings (ratings must be between 1 and 5)
df_ratings_clean = df_ratings_clean.filter((col("rating") >= 0) & (col("rating") <= 5))

# Remove rows with NULL values
df_ratings_clean = df_ratings_clean.na.drop()

# Print count after cleaning
print("Total Number of Rows in the Ratings dataset after cleaning:", df_ratings_clean.count())

# Show a sample of cleaned data
df_ratings_clean.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Total Number of Rows in the Ratings dataset after cleaning: 5152656
+-------+------+-------+
|item_id|rating|user_id|
+-------+------+-------+
|4790821|     5| 215684|
|1997473|     4|  40815|
|2183601|     5| 215920|
|1241552|     5| 216463|
|1812101|     4| 130245|
+-------+------+-------+
only showing top 5 rows

# Storing data in AWS Athena

In [11]:
# Save the cleaned DataFrame as a CSV file in S3
S3_BUCKET_NAME = 'book-recommendation-project'
S3_KEY = 'data1/ratings_cleaned.csv'

df_ratings_clean.write.option("header", "true").csv(f"s3a://{S3_BUCKET_NAME}/{S3_KEY}")

print(f"Data saved to: s3://{S3_BUCKET_NAME}/{S3_KEY}")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Data saved to: s3://book-recommendation-project/data1/ratings_cleaned.csv

In [ ]:
import boto3

# Athena configuration
athena_client = boto3.client('athena',
    aws_access_key_id='your keys', #your access key
    aws_secret_access_key='your keys', #your secret access key
    aws_session_token='your keys', #your session token
    region_name = 'us-east-1')

# Define the table creation query for CSV format
DATABASE_NAME = 'default'
TABLE_NAME = 'book_ratings_table3'

create_table_query = f"""
CREATE EXTERNAL TABLE IF NOT EXISTS `{DATABASE_NAME}`.`{TABLE_NAME}` (
    user_id INT,
    item_id INT,
    rating INT
)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY ','
LOCATION 's3://{S3_BUCKET_NAME}/{S3_KEY}'
"""

# Execute query to create the table
response = athena_client.start_query_execution(
    QueryString=create_table_query.strip(),
    QueryExecutionContext={'Database': DATABASE_NAME},
    ResultConfiguration={'OutputLocation': f's3://{S3_BUCKET_NAME}/query_results/'}
)

# Wait and check query status
query_execution_id = response['QueryExecutionId']
response = athena_client.get_query_execution(QueryExecutionId=query_execution_id)
status = response["QueryExecution"]["Status"]["State"]
print(f"Query Status: {status}")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Query Status: QUEUED

In [17]:
#Query the first 10 rows from the S3 Using Athena SQL query

query = """
    SELECT *
    FROM book_ratings_table3
    LIMIT 10
"""

# Start the query execution in Athena
response = athena_client.start_query_execution(
    QueryString=query,
    QueryExecutionContext={'Database': 'default'},  # Replace with your database name
    ResultConfiguration={'OutputLocation': f's3://{S3_BUCKET_NAME}/query_results/'}
)

# Get the query execution ID
query_execution_id = response['QueryExecutionId']

# Poll Athena until the query is completed
response = athena_client.get_query_execution(QueryExecutionId=query_execution_id)
status = response["QueryExecution"]["Status"]["State"]
print(status)



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

QUEUED

In [21]:
def get_query_results_as_dataframe(query_execution_id,spark=spark):
    # Fetching the query results
    results_paginator = athena_client.get_paginator('get_query_results')
    results_iter = results_paginator.paginate(QueryExecutionId=query_execution_id)

    # Initialize a list to store rows and columns for DataFrame
    columns = []
    rows = []
    
    # Process the result pages
    for results_page in results_iter:
        # Get column info from the first page
        if not columns:
            columns = [col['Label'] for col in results_page['ResultSet']['ResultSetMetadata']['ColumnInfo']]
            print(columns)
        # Skip the first row of the first page (column headers)
        for row in results_page['ResultSet']['Rows'][2:]:
            rows.append([col.get('VarCharValue', None) for col in row['Data']])
    
    import pandas as pd
#     # Create DataFrame from the results
    queried_df = pd.DataFrame(rows, columns=columns)
    
#     Count the number of 7-digit values in each column
    column_lengths = queried_df.applymap(lambda x: len(str(x)))

    # Define the condition for majority of length being 7
    majority_7 = column_lengths.apply(lambda col: (col == 7).sum() > len(col) // 2)

    # Rename columns based on the length conditions
    queried_df.columns = ['rating' if queried_df[col].apply(lambda x: len(str(x)) == 1).all() else ('item_id' if majority_7[col] else 'user_id') for col in queried_df.columns]
    
    return queried_df

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
response = athena_client.get_query_execution(QueryExecutionId=query_execution_id)
status = response["QueryExecution"]["Status"]["State"]
print(status)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SUCCEEDED

In [23]:
queried_df = get_query_results_as_dataframe(query_execution_id)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['user_id', 'item_id', 'rating']
<stdin>:25: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.

In [24]:
queried_df

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

    item_id rating user_id
0   7350661      1   60896
1   7351419      5   35712
2   7351419      3   33291
3   7352929      3   80508
4   7356968      5  259734
5   6596168      4   17676
6  10424899      3   67985
7   7374960      5   86317
8   7377150      2  123829

In [101]:
# data = []
# columns =[ i['VarCharValue'] for i in results['ResultSet']['Rows'][0]['Data']]
# for i in results['ResultSet']['Rows'][2:]:
# #     print(i['Data'])
#     rows =[j['VarCharValue'] for j in i['Data']]
#     data.append(rows)

# queried_df = pd.DataFrame(data, columns=columns)
# queried_df

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Exploratory Data Analysis

In [174]:
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql.functions import col, avg

# Aggregate to get average rating for each item_id
top_liked_books = df_ratings.groupBy("item_id").agg(avg("rating").alias("avg_rating"))

# Convert to Pandas for visualization
top_liked_books_pd = top_liked_books.orderBy(col("avg_rating").desc()).limit(10).toPandas()
print(top_liked_books_pd)

# Plot top 10 liked books
plt.figure(figsize=(12, 6))
sns.barplot(x="item_id", y="avg_rating", data=top_liked_books_pd, hue="item_id", palette="viridis", legend=False)
plt.xticks(rotation=45)
plt.xlabel("Book ID")
plt.ylabel("Average Rating")
plt.title("Top 10 Most Liked Books")
plt.show()
# Save plot as an image
plot_path = "top_10_liked_books.png"
plt.savefig(plot_path, bbox_inches="tight")  # Save the plot
plt.close()  # Close to free memory

# Provide a download link
display(FileLink(plot_path))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

    item_id  avg_rating
0  26518929    4.826568
1  52922245    4.818605
2  49704739    4.801546
3  41343043    4.788571
4   2962492    4.786963
5  28323940    4.780161
6  26419682    4.770270
7  47472732    4.770000
8  51859542    4.768750
9  16482835    4.767677
/mnt1/yarn/usercache/livy/appcache/application_1742334962508_0003/container_1742334962508_0003_01_000001/top_10_liked_books.png

# Top 10 Most Liked Books

In [182]:
# Aggregate to get average rating for each item_id
top_liked_books = df_ratings.groupBy("item_id").agg(avg("rating").alias("avg_rating"))

# Convert to Pandas for visualization
top_liked_books_pd = top_liked_books.orderBy(col("avg_rating").desc()).limit(10).toPandas()
print(top_liked_books_pd)
# Plot top 10 liked books
plt.figure(figsize=(12, 6))
sns.barplot(x="item_id", y="avg_rating", data=top_liked_books_pd, palette="viridis")
plt.xticks(rotation=45)
plt.xlabel("Book ID")
plt.ylabel("Average Rating")
plt.title("Top 10 Most Liked Books")
plt.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<stdin>:9: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

# Top 10 Least Liked Books

In [ ]:
# Aggregate to get lowest-rated books
least_liked_books = df_ratings.groupBy("item_id").agg(avg("rating").alias("avg_rating"))

# Convert to Pandas for visualization
least_liked_books_pd = least_liked_books.orderBy(col("avg_rating").asc()).limit(10).toPandas()
print(least_liked_books_pd)
# Plot least liked books
plt.figure(figsize=(12, 6))
sns.barplot(x="item_id", y="avg_rating", data=least_liked_books_pd, palette="coolwarm")
plt.xticks(rotation=45)
plt.xlabel("Book ID")
plt.ylabel("Average Rating")
plt.title("Top 10 Least Liked Books")
plt.show()


# Most Reviewed Books

In [ ]:
most_reviewed_books = df_ratings.groupBy("item_id").agg(count("rating").alias("rating_count"))
most_reviewed_books_pd = most_reviewed_books.orderBy(col("rating_count").desc()).limit(10).toPandas()

plt.figure(figsize=(12, 6))
sns.barplot(x="item_id", y="rating_count", data=most_reviewed_books_pd, palette="magma")
plt.xticks(rotation=45)
plt.xlabel("Book ID")
plt.ylabel("Number of Ratings")
plt.title("Top 10 Most Reviewed Books")
plt.show()

#  Training a Recommendation System

In [25]:
sc.install_pypi_package("numpy")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
Package already installed for current Spark context!
Traceback (most recent call last):
  File "/mnt1/yarn/usercache/livy/appcache/application_1742356023609_0001/container_1742356023609_0001_01_000001/pyspark.zip/pyspark/context.py", line 2614, in install_pypi_package
    raise ValueError("Package already installed for current Spark context!")
ValueError: Package already installed for current Spark context!



In [26]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col

# Split the data into training and validation sets (80% train, 20% validation)
train_data, validation_data = df_ratings_clean.randomSplit([0.8, 0.2], seed=42)

#  Define the ALS model
als = ALS(userCol="user_id", itemCol="item_id", ratingCol="rating", coldStartStrategy="drop")

# Set up cross-validation (without parameter grid)
crossval = CrossValidator(
    estimator=als,
    estimatorParamMaps=ParamGridBuilder().build(),  # No parameter grid, just ALS with default params
    evaluator=RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction"),
    numFolds=5  # 5-fold cross-validation
)

# Fit the model with cross-validation on the training data
cv_model = crossval.fit(train_data)

# Get the best model from cross-validation
best_model = cv_model.bestModel

# Make predictions on the validation set
predictions = best_model.transform(validation_data)

# Evaluate the model on the validation data
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
rmse = evaluator.evaluate(predictions)

print(f"RMSE on validation data: {rmse}")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-26:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.9/threading.py", line 980, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.9/threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.9/site-packages/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.9/site-packages/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'


RMSE on validation data: 0.9535117233441551

In [27]:
# Generate top-10 recommendations for all users
recommendations = best_model.recommendForAllUsers(10)
recommendations.show()



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-27:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.9/threading.py", line 980, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.9/threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.9/site-packages/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.9/site-packages/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'


+-------+--------------------+
|user_id|     recommendations|
+-------+--------------------+
|     12|[{2207992, 5.8894...|
|     65|[{2962492, 5.0170...|
|     76|[{44749881, 5.445...|
|     81|[{2962492, 4.1358...|
|    122|[{649656, 5.21488...|
|    126|[{905641, 5.10007...|
|    133|[{28323940, 5.323...|
|    140|[{836462, 4.63607...|
|    148|[{44600531, 4.573...|
|    177|[{905641, 5.12008...|
|    192|[{16482835, 4.808...|
|    209|[{2139868, 5.3686...|
|    243|[{51438471, 4.618...|
|    300|[{878114, 4.73622...|
|    333|[{49704739, 4.358...|
|    406|[{44600531, 4.557...|
|    417|[{44749881, 5.489...|
|    444|[{49638190, 4.823...|
|    481|[{310090, 5.20069...|
|    496|[{6801520, 4.8514...|
+-------+--------------------+
only showing top 20 rows

In [29]:
data = recommendations.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-29:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.9/threading.py", line 980, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.9/threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.9/site-packages/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.9/site-packages/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'


+-------+--------------------+
|user_id|     recommendations|
+-------+--------------------+
|     12|[{2207992, 5.8894...|
|     65|[{2962492, 5.0170...|
|     76|[{44749881, 5.445...|
|     81|[{2962492, 4.1358...|
|    122|[{649656, 5.21488...|
|    126|[{905641, 5.10007...|
|    133|[{28323940, 5.323...|
|    140|[{836462, 4.63607...|
|    148|[{44600531, 4.573...|
|    177|[{905641, 5.12008...|
|    192|[{16482835, 4.808...|
|    209|[{2139868, 5.3686...|
|    243|[{51438471, 4.618...|
|    300|[{878114, 4.73622...|
|    333|[{49704739, 4.358...|
|    406|[{44600531, 4.557...|
|    417|[{44749881, 5.489...|
|    444|[{49638190, 4.823...|
|    481|[{310090, 5.20069...|
|    496|[{6801520, 4.8514...|
+-------+--------------------+
only showing top 20 rows